# Actividad N° 05: iFood

## Integrantes

**Grupo N° 03**

- Adriana Villalobos
- Gustavo Ledesma
- Alejo Cuello

## Descripción de la actividad

Trabajamos sobre el conjunto de datos *aw_fb_data.csv* de Apple Watch y Fitbit Data. El objetivo de la actividad aplicar clustering para agrupar los datos y describir cada uno de estos clusters.

# Consigna

**Consideraciones**

- Nos basamos en el dataset de Kaggle: Apple Watch and Fitbit data.
- Repliquen el notebook y apliquen lo aprendido a este dataset.

**Preparación de datos**

- Eliminar la columna X1 y hacer un get dummies.

**KMeans**

- Probar con n_clusters=3.
- Gráfico 3D para las columnas x='gender', y='age', z='calories' y color='activity'

**HDBSCAN**

- Probar con min_cluster_size=80.

**Output**

- Hagan un storytelling para 3 cluster que les llamo la atención, tanto para KMeans y HDBSCAN.

# Código

## Importación de librerías y datos

In [13]:
import pandas as pd
from funpymodeling.exploratory import status
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv("aw_fb_data.csv", sep=",", index_col=0)
data.head(3)

,X1,age,gender,height,weight,steps,hear_rate,calories,distance,entropy_heart,entropy_setps,resting_heart,corr_heart_steps,norm_heart,intensity_karvonen,sd_norm_heart,steps_times_distance,device,activity
1,1,20,1,168.0,65.4,10.771429,78.531302,0.344533,0.008327,6.221612,6.116349,59.0,1.0,19.531302,0.138520,1.0,0.089692,apple watch,Lying
2,2,20,1,168.0,65.4,11.475325,78.453390,3.287625,0.008896,6.221612,6.116349,59.0,1.0,19.453390,0.137967,1.0,0.102088,apple watch,Lying
3,3,20,1,168.0,65.4,12.179221,78.540825,9.484000,0.009466,6.221612,6.116349,59.0,1.0,19.540825,0.138587,1.0,0.115287,apple watch,Lying


## Análisis inicial de los datos

In [6]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,X1,0,0.0,0,0.000000,3656,int64
1,age,0,0.0,0,0.000000,24,int64
2,gender,0,0.0,3279,0.523467,2,int64
3,height,0,0.0,0,0.000000,28,float64
4,weight,0,0.0,0,0.000000,43,float64
5,steps,0,0.0,0,0.000000,3919,float64
6,hear_rate,0,0.0,0,0.000000,4514,float64
7,calories,0,0.0,0,0.000000,2136,float64
8,distance,0,0.0,0,0.000000,4863,float64
9,entropy_heart,0,0.0,18,0.002874,56,float64


## Preparación inicial de los datos

In [7]:
data.drop(columns=["X1"], inplace=True)

In [15]:
data = pd.get_dummies(data)
data.head(3)

,age,gender,height,weight,steps,hear_rate,calories,distance,entropy_heart,entropy_setps,...,sd_norm_heart,steps_times_distance,device_apple watch,device_fitbit,activity_Lying,activity_Running 3 METs,activity_Running 5 METs,activity_Running 7 METs,activity_Self Pace walk,activity_Sitting
1,20,1,168.0,65.4,10.771429,78.531302,0.344533,0.008327,6.221612,6.116349,...,1.0,0.089692,True,False,True,False,False,False,False,False
2,20,1,168.0,65.4,11.475325,78.453390,3.287625,0.008896,6.221612,6.116349,...,1.0,0.102088,True,False,True,False,False,False,False,False
3,20,1,168.0,65.4,12.179221,78.540825,9.484000,0.009466,6.221612,6.116349,...,1.0,0.115287,True,False,True,False,False,False,False,False


In [16]:
std_scaler = StandardScaler()
std_scaler.fit(data)
x_data_norm = std_scaler.transform(data)

pd.DataFrame(x_data_norm).describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,...,6.264000e+03,6.264000e+03,6264.000000,6264.000000,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03,6.264000e+03
mean,2.268655e-17,2.177909e-16,-5.172533e-16,3.720594e-16,1.814924e-16,9.074620e-17,1.814924e-16,-1.814924e-17,-9.165366e-16,7.985665e-16,...,3.629848e-17,-9.074620e-18,0.000000,0.000000,-3.232833e-17,-2.608953e-17,-2.949251e-17,4.991041e-17,-3.913430e-17,5.671637e-18
std,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,...,1.000080e+00,1.000080e+00,1.000080,1.000080,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00,1.000080e+00
min,-1.252603e+00,-9.541166e-01,-2.587115e+00,-1.978652e+00,-4.873068e-01,-2.929548e+00,-7.109949e-01,-3.011055e-01,-7.877479e+00,-4.569150e+00,...,-6.471041e-01,-1.452030e-01,-1.183993,-0.844599,-5.313123e-01,-4.228156e-01,-4.363736e-01,-4.650921e-01,-4.066883e-01,-4.175562e-01
25%,-6.913270e-01,-9.541166e-01,-9.404466e-01,-7.147873e-01,-4.686358e-01,-3.680869e-01,-6.861079e-01,-3.006985e-01,1.021668e-01,1.348908e-01,...,-6.259839e-01,-1.450410e-01,-1.183993,-0.844599,-5.313123e-01,-4.228156e-01,-4.363736e-01,-4.650921e-01,-4.066883e-01,-4.175562e-01
50%,-1.300505e-01,-9.541166e-01,-1.655436e-01,-1.200273e-01,-4.464951e-01,-3.098031e-01,-5.665760e-01,-2.971593e-01,2.083702e-01,3.321103e-01,...,-4.162532e-01,-1.419133e-01,0.844599,-0.844599,-5.313123e-01,-4.228156e-01,-4.363736e-01,-4.650921e-01,-4.066883e-01,-4.175562e-01
75%,4.312259e-01,1.048090e+00,9.968107e-01,5.713812e-01,-1.667584e-02,3.325684e-01,3.765169e-02,4.059042e-02,2.842709e-01,4.043337e-01,...,1.251642e-01,-1.221373e-01,0.844599,1.183993,-5.313123e-01,-4.228156e-01,-4.363736e-01,-4.650921e-01,-4.066883e-01,-4.175562e-01
max,3.013097e+00,1.048090e+00,2.062302e+00,3.374187e+00,7.201889e+00,3.776815e+00,2.857381e+00,6.991359e+00,5.818567e-01,5.856720e-01,...,5.293335e+00,1.253348e+01,0.844599,1.183993,1.882132e+00,2.365097e+00,2.291614e+00,2.150112e+00,2.458886e+00,2.394887e+00


## Clustering

### KMeans

### HDBSCAN